In [1]:
#Only once when launching notebook
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')

import matplotlib.pyplot as plt
from time import time
%matplotlib widget
print("Config loaded")

Config loaded


# Load Data Info

In [10]:
import mne
from tinnsleep.config import Config
from tinnsleep.data import CreateRaw
import pandas as pd
data_info = pd.read_csv("data/data_info.csv",engine='python', sep=";")
print(data_info[data_info["mema"]==1]["filename"])

4          1DA15_nuit_hab.edf
9          1GB18_nuit_hab.edf
22         1RA17_nuit_hab.edf
28           1HB20_nuit_1.edf
29           1HB20_nuit_2.edf
31           1SL21_nuit_4.edf
32           1SL21_nuit_6.edf
33           1SL21_nuit_7.edf
34           1SL21_nuit_8.edf
36           1SL21_nuit_2.edf
37           1SL21_nuit_3.edf
38           1SL21_nuit_5.edf
39           1UC22_nuit_2.edf
40           1UC22_nuit_1.edf
42           2GA06_nuit_1.edf
43           2SJ03_nuit_1.edf
45           2GR01_nuit_3.edf
46           2GR01_nuit_4.edf
49           2LS05_nuit_1.edf
50           2CT04_nuit_1.edf
51    1HB20_nuit_1_resmed.edf
52    1HB20_nuit_2_resmed.edf
53    1SL21_nuit_2_resmed.edf
54    1SL21_nuit_3_resmed.edf
55    1UC22_nuit_1_resmed.edf
56    1UC22_nuit_2_resmed.edf
Name: filename, dtype: object


# Select data

In [16]:
filename = Config.bruxisme_files[4]  # load file from config
mema_files = data_info[data_info["mema"]==1]['filename'].values
assert filename.split(os.path.sep)[-1] in mema_files, f"MEMA not available for {filename.split(os.path.sep)[-1]}"
picks_chan = ['Airflow']           # subset of EMG electrodes
ch_types = ["misc"]

start = time()

raw  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading
tmin = raw.times[0]                     
tmax = raw.times[-1]
raw  = CreateRaw(raw[picks_chan][0], raw.info["sfreq"], picks_chan, ch_types=ch_types)        # pick channels and load

raw  = raw.filter(0.1, 5., n_jobs=4, 
                  fir_design='firwin', filter_length='auto', phase='zero-double',
                  picks=['Airflow'])
ch_names = raw.info["ch_names"]
print(f"Data filtered ({time()-start:.2f}s)")
offset = raw.times[0]
print(f"keeping {(raw.times[-1]-raw.times[0])/3600:0.2f} hours of recording out of {(tmax-tmin)/3600:0.2f} hours")

Extracting EDF parameters from /Users/louis/Data/SIOPI/bruxisme/1DA15_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-16-510d529b7a7d>:9: RuntimeWarning: 6 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading


Data filtered (2.18s)
keeping 8.17 hours of recording out of 8.17 hours
